In [1]:
using DataFrames, Plots, CSV, Dates, LaTeXStrings, CmdStan, cmdstan_utils, Statistics, LinearAlgebra

┌ Info: Precompiling cmdstan_utils [b681c197-c997-42fd-b5bb-d7d7839f617e]
└ @ Base loading.jl:1260
┌ Warning: Package cmdstan_utils does not have DataFrames in its dependencies:
│ - If you have cmdstan_utils checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with cmdstan_utils
└ Loading DataFrames into cmdstan_utils from project dependency, future warnings for cmdstan_utils are suppressed.


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10614024257914604326\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5693445580229629234\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10614024257914604326\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10395436131720218452\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15759105593523613277\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10395436131720218452\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Extracting file

In [2]:
file = "data/20200617_r1_O1_T_tetracycline_growth.csv"

"data/20200617_r1_O1_T_tetracycline_growth.csv"

In [3]:
include("src/data_handling.jl")
include("src/viz.jl")
default_plotlyjs!()

df = read_OD(file);


In [4]:
plot()
for i in 15:15
    plot!(
        df[df.Well .== Symbol("Column$i"), Symbol("time_[s]")], 
        df[df.Well .== Symbol("Column$i"), :OD], 
        label=:none,
        xlabel="time [s]",
        ylabel = L"OD_{600}"
    )
end
plot!()

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
t = df[df.Well .== Symbol("Column15"), Symbol("time_[s]")]*1.
N = df[df.Well .== Symbol("Column15"), :OD];

## Maximum Likelihood Gaussian Process

In [6]:
mle_stan_file = "
data {
  int<lower=1> N;
  real x[N];
  vector[N] y;
}

parameters {
  real<lower=0> rho;
  real<lower=0> alpha;
  real<lower=0> sigma;
}

model {
  matrix[N, N] cov =   cov_exp_quad(x, alpha, rho)
                     + diag_matrix(rep_vector(square(sigma), N));
  matrix[N, N] L_cov = cholesky_decompose(cov);

  y ~ multi_normal_cholesky(rep_vector(0, N), L_cov);
}"

"\ndata {\n  int<lower=1> N;\n  real x[N];\n  vector[N] y;\n}\n\nparameters {\n  real<lower=0> rho;\n  real<lower=0> alpha;\n  real<lower=0> sigma;\n}\n\nmodel {\n  matrix[N, N] cov =   cov_exp_quad(x, alpha, rho)\n                     + diag_matrix(rep_vector(square(sigma), N));\n  matrix[N, N] L_cov = cholesky_decompose(cov);\n\n  y ~ multi_normal_cholesky(rep_vector(0, N), L_cov);\n}"

In [7]:
mle_data=Dict(
    "N"=>length(t),
    "x"=>t,
    "y"=>N,
)
mle_model = Stanmodel(
  name="mle_test", 
  nchains=4,
  num_warmup=1000, 
  num_samples=1000,
  thin=10,
  model=mle_stan_file,
  printsummary=false,
)


File /Users/tomroschinger/git/evo_mwc_julia/tmp/mle_test.stan will be updated.



  name =                    "mle_test"
  nchains =                 4
  num_samples =             1000
  num_warmup =                1000
  thin =                    10
  monitors =                String[]
  model_file =              "mle_test.stan"
  data_file =               ""
  output =                  Output()
    file =                    ""
    diagnostics_file =        ""
    refresh =                 100
  pdir =                   "/Users/tomroschinger/git/evo_mwc_julia"
  tmpdir =                 "/Users/tomroschinger/git/evo_mwc_julia/tmp"
  output_format =           :array
  method =                  Sample()
    num_samples =             1000
    num_warmup =              1000
    save_warmup =             false
    thin =                    10
    algorithm =               HMC()
      engine =                  NUTS()
        max_depth =               10
      metric =                  CmdStan.diag_e
      stepsize =                1.0
      stepsize_jitter =         1.0
 

In [8]:
a, mle_chains, mle_b = stan(
  mle_model, 
  mle_data, 
  summary=false
)

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: cholesky_decompose: Matrix A is not positive definite (in '/Users/tomroschinger/git/evo_mwc_julia/tmp/mle_test.stan', line 16, column 2 to column 47)
If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: cholesky_decompose: Matrix A is not positive definite (in '/Users/tomroschinger/git/evo_mwc_julia/tmp/mle_test.stan', line 16, column 2 to column 47)
If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-cond

(0, [296.907 0.901132 … 0.368107 0.00233105; 296.851 0.994729 … 0.485267 0.0033521; … ; 297.588 0.954649 … 0.401803 0.00273384; 296.566 0.986244 … 0.477669 0.00247872]

[297.217 0.954318 … 0.362541 0.00311902; 296.621 0.918627 … 0.441701 0.00246079; … ; 296.004 0.9275 … 0.567963 0.00341375; 295.209 0.787715 … 0.428758 0.00273026]

[296.994 0.990881 … 0.341736 0.00284679; 297.091 0.973885 … 0.505307 0.00287528; … ; 296.373 0.878628 … 0.388771 0.00248498; 296.908 0.941765 … 0.46432 0.00247934]

[291.973 0.798569 … 0.301814 0.00333668; 297.246 0.92763 … 0.38044 0.00305058; … ; 296.395 0.839211 … 0.525122 0.00253829; 297.538 0.870283 … 0.37552 0.00259537], ["lp__", "accept_stat__", "stepsize__", "treedepth__", "n_leapfrog__", "divergent__", "energy__", "rho", "alpha", "sigma"])

In [9]:
d = collect_params_from_chain(mle_b, mle_chains[:,:,1])

Dict{Any,Any} with 10 entries:
  "treedepth__"   => [2.0; 2.0; … ; 2.0; 3.0]
  "n_leapfrog__"  => [3.0; 3.0; … ; 7.0; 7.0]
  "sigma"         => [0.00233105; 0.0033521; … ; 0.00273384; 0.00247872]
  "energy__"      => [-296.111; -295.823; … ; -297.362; -295.145]
  "lp__"          => [296.907; 296.851; … ; 297.588; 296.566]
  "alpha"         => [0.368107; 0.485267; … ; 0.401803; 0.477669]
  "accept_stat__" => [0.901132; 0.994729; … ; 0.954649; 0.986244]
  "divergent__"   => [0.0; 0.0; … ; 0.0; 0.0]
  "stepsize__"    => [0.509089; 0.509089; … ; 0.509089; 0.509089]
  "rho"           => [6408.13; 8224.2; … ; 6911.87; 7462.68]

In [10]:
println(mean(d["rho"]))
println(mean(d["alpha"]))
println(mean(d["sigma"]))


7797.490900000001
0.44596404
0.0030189472


## Simulation from gaussian process

In [11]:
pred_gauss_file = "
functions {
  vector gp_pred_rng(real[] x2,
                     vector y1, real[] x1,
                     real alpha, real rho, real sigma, real delta) {
    int N1 = rows(y1);
    int N2 = size(x2);
    vector[N2] f2;
    {
      matrix[N1, N1] K =   cov_exp_quad(x1, alpha, rho)
                         + diag_matrix(rep_vector(square(sigma), N1));
      matrix[N1, N1] L_K = cholesky_decompose(K);

      vector[N1] L_K_div_y1 = mdivide_left_tri_low(L_K, y1);
      vector[N1] K_div_y1 = mdivide_right_tri_low(L_K_div_y1', L_K)';
      matrix[N1, N2] k_x1_x2 = cov_exp_quad(x1, x2, alpha, rho);
      vector[N2] f2_mu = (k_x1_x2' * K_div_y1);
      matrix[N1, N2] v_pred = mdivide_left_tri_low(L_K, k_x1_x2);
      matrix[N2, N2] cov_f2 =   cov_exp_quad(x2, alpha, rho) - v_pred' * v_pred
                              + diag_matrix(rep_vector(delta, N2));
      f2 = multi_normal_rng(f2_mu, cov_f2);
    }
    return f2;
  }
}

data {
  int<lower=1> N;
  real x[N];
  vector[N] y;

  int<lower=1> N_predict;
  real x_predict[N_predict];

  real<lower=0> rho;
  real<lower=0> alpha;
  real<lower=0> sigma;
}

transformed data {
  matrix[N, N] cov =   cov_exp_quad(x, alpha, rho)
                     + diag_matrix(rep_vector(1e-10, N));
  matrix[N, N] L_cov = cholesky_decompose(cov);
}

parameters {}
model {}

generated quantities {
  vector[N_predict] f_predict = gp_pred_rng(x_predict, y, x, alpha, rho, sigma, 1e-10);
  vector[N_predict] y_predict;

  for (n in 1:N_predict)
    y_predict[n] = normal_rng(f_predict[n], sigma);
}"

"\nfunctions {\n  vector gp_pred_rng(real[] x2,\n                     vector y1, real[] x1,\n                     real alpha, real rho, real sigma, real delta) {\n    int N1 = rows(y1);\n    int N2 = size(x2);\n    vector[N2] f2;\n    {\n      matrix[N1, N1] K =   cov_exp_quad(x1, alpha, rho)\n                         + diag_matrix(rep_vector(square(sigma), N1));\n      matrix[N1, N1] L_K = cholesky_decompose(K);\n\n      vector[N1] L_K_div_y1 = mdivide_left_tri_low(L_K, y1);\n      vector[N1] K_div_y1 = mdivide_right_tri_low(L_K_div_y1', L_K)';\n      matrix[N1, N2] k_x1_x2 = cov_exp_quad(x1, x2, alpha, rho);\n      vector[N2] f2_mu = (k_x1_x2' * K_div_y1);\n      matrix[N1, N2] v_pred = mdivide_left_tri_low(L_K, k_x1_x2);\n      matrix[N2, N2] cov_f2 =   cov_exp_quad(x2, alpha, rho) - v_pred' * v_pred\n                              + diag_matrix(rep_vector(delta, N2));\n      f2 = multi_normal_rng(f2_mu, cov_f2);\n    }\n    return f2;\n  }\n}\n\ndata {\n  int<lower=1> N;\n  real x[N

In [12]:
pred_gauss_model = Stanmodel(
  name="pred_gauss", 
  nchains=1,
  num_warmup=0, 
  num_samples=1000,
  model=pred_gauss_file,
  printsummary=false,
  Sample(algorithm=CmdStan.Fixed_param())
)

pred_gauss_data = Dict(    
    "N"=>length(t),
    "x"=>t,
    "y"=>N,
    "N_predict"=>500,
    "x_predict"=>range(minimum(t), stop=maximum(t), length=500),
    "rho" => mean(d["rho"]),
    "alpha" => mean(d["alpha"]),
    "sigma" => mean(d["sigma"])
)
    

_, pred_gauss_chains, pred_gauss_names = stan(pred_gauss_model, pred_gauss_data, summary=false);


File /Users/tomroschinger/git/evo_mwc_julia/tmp/pred_gauss.stan will be updated.



In [13]:
d = collect_params_from_chain(pred_gauss_names, pred_gauss_chains[:,:,1])

Dict{Any,Any} with 4 entries:
  "f_predict"     => [0.0379793 0.0386109 … 0.82247 0.822238; 0.036378 0.036510…
  "lp__"          => [0.0; 0.0; … ; 0.0; 0.0]
  "accept_stat__" => [0.0; 0.0; … ; 0.0; 0.0]
  "y_predict"     => [0.0356496 0.0456165 … 0.821097 0.817401; 0.0375269 0.0377…

In [14]:
y = [d["y_predict"][i, :] for i in 1:1000];

In [15]:
predictive_regression(y, range(minimum(t), stop=maximum(t), length=500), data=[t, N])

<!DOCTYPE html>
 
 
 Plots.jl

## Bayesian workflow

In [104]:
bw_file="
functions {
    vector gp_pred_rng(real[] x2,
                         vector y1, real[] x1,
                         real alpha, real rho, real sigma, real delta) {
        int N1 = rows(y1);
        int N2 = size(x2);
        vector[N2] f2;
        {
          matrix[N1, N1] K =   cov_exp_quad(x1, alpha, rho)
                             + diag_matrix(rep_vector(square(sigma), N1));
          matrix[N1, N1] L_K = cholesky_decompose(K);

          vector[N1] L_K_div_y1 = mdivide_left_tri_low(L_K, y1);
          vector[N1] K_div_y1 = mdivide_right_tri_low(L_K_div_y1', L_K)';
          matrix[N1, N2] k_x1_x2 = cov_exp_quad(x1, x2, alpha, rho);
          vector[N2] f2_mu = (k_x1_x2' * K_div_y1);
          matrix[N1, N2] v_pred = mdivide_left_tri_low(L_K, k_x1_x2);
          matrix[N2, N2] cov_f2 =   cov_exp_quad(x2, alpha, rho) - v_pred' * v_pred
                                  + diag_matrix(rep_vector(delta, N2));
          f2 = multi_normal_rng(f2_mu, cov_f2);
        }
        return f2;
      }

  vector gp_pred_der_rng(real[] x2,
                     vector y1, real[] x1,
                     real alpha, real rho, real sigma, real delta) {
    int N1 = rows(y1);
    int N2 = size(x2);
    vector[N2] g2;
    {     
      matrix[N1, N1] K =   cov_exp_quad(x1, alpha, rho)
                             + diag_matrix(rep_vector(square(sigma), N1));

      matrix[N1, N1] L_K = cholesky_decompose(K);

      vector[N1] L_K_div_y1 = mdivide_left_tri_low(L_K, y1);
      vector[N1] K_div_y1 = mdivide_right_tri_low(L_K_div_y1', L_K)';
      matrix[N1, N2] k_x1_x2 = cov_exp_quad(x1, x2, alpha, rho);
      
      matrix[N1, N2] K_1;
      for (i in 1:N1){
        for (j in 1:N2){
          K_1[i, j] = (x1[i] - x2[j])/rho^2 * k_x1_x2[i,j];
        }
      }
      matrix[N2, N2] k_x2_x2 = cov_exp_quad(x2, alpha, rho);
      matrix[N2, N2] K_2;
      for (i in 1:N2){
        for (j in 1:N2){
          K_2[i, j] = (1/rho^2 - (x2[i] - x2[j])^2 /rho^4) * k_x2_x2[i, j];
        }
      }
      matrix[N1, N2] v_pred = mdivide_left_tri_low(L_K, K_1);
      vector[N2] g2_mu = (K_1' * K_div_y1);
      matrix[N2, N2] cov_g2 =   K_2 - v_pred' * v_pred
                              + diag_matrix(rep_vector(delta, N2));

      g2 = multi_normal_rng(g2_mu, cov_g2);
    }
    return g2;
  }
}

data {
  int<lower=1> N;
  real x[N];
  vector[N] y;

  int<lower=1> N_predict;
  real x_predict[N_predict];
}

parameters {
  real<lower=0> rho;
  real<lower=0> alpha;
  real<lower=0> sigma;
}

model {
  matrix[N, N] cov =   cov_exp_quad(x, alpha, rho)
                     + diag_matrix(rep_vector(square(sigma), N));
  matrix[N, N] L_cov = cholesky_decompose(cov);

  // P[rho < 2.0] = 0.01
  // P[rho > 10] = 0.01
  rho ~ normal(8000, 500);
  alpha ~ normal(0, 2);
  sigma ~ normal(0, 1);

  y ~ multi_normal_cholesky(rep_vector(0, N), L_cov);
}

generated quantities {
  vector[N_predict] f_predict = gp_pred_rng(x_predict, y, x, alpha, rho, sigma, 1e-10);
  vector[N_predict] g_predict = gp_pred_der_rng(x_predict, y, x, alpha, rho, sigma, 1e-10);
  vector[N_predict] y_predict;
  for (n in 1:N_predict){
    y_predict[n] = normal_rng(f_predict[n], sigma);
}
}

"



"\nfunctions {\n    vector gp_pred_rng(real[] x2,\n                         vector y1, real[] x1,\n                         real alpha, real rho, real sigma, real delta) {\n        int N1 = rows(y1);\n        int N2 = size(x2);\n        vector[N2] f2;\n        {\n          matrix[N1, N1] K =   cov_exp_quad(x1, alpha, rho)\n                             + diag_matrix(rep_vector(square(sigma), N1));\n          matrix[N1, N1] L_K = cholesky_decompose(K);\n\n          vector[N1] L_K_div_y1 = mdivide_left_tri_low(L_K, y1);\n          vector[N1] K_div_y1 = mdivide_right_tri_low(L_K_div_y1', L_K)';\n          matrix[N1, N2] k_x1_x2 = cov_exp_quad(x1, x2, alpha, rho);\n          vector[N2] f2_mu = (k_x1_x2' * K_div_y1);\n          matrix[N1, N2] v_pred = mdivide_left_tri_low(L_K, k_x1_x2);\n          matrix[N2, N2] cov_f2 =   cov_exp_quad(x2, alpha, rho) - v_pred' * v_pred\n                                  + diag_matrix(rep_vector(delta, N2));\n          f2 = multi_normal_rng(f2_mu, cov_f2);\n

In [105]:
bw_model = Stanmodel(
  name="bw", 
  nchains=4,
  num_warmup=5000,
  num_samples=5000,
  thin=50,
  model=bw_file,
  printsummary=false
)

bw_data = Dict(    
    "N"=>length(t),
    "x"=>t,
    "y"=>N,
    "N_predict"=>500,
    "x_predict"=>range(minimum(t), stop=maximum(t), length=500),
)
    

_, bw_chains, bw_names = stan(bw_model, bw_data, summary=false);


File /Users/tomroschinger/git/evo_mwc_julia/tmp/bw.stan will be updated.



Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: gp_exp_quad_cov: sigma is 0, but must be > 0! (in '/Users/tomroschinger/git/evo_mwc_julia/tmp/bw.stan', line 81, column 2 to line 82, column 65)
If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: cholesky_decompose: Matrix A is not positive definite (in '/Users/tomroschinger/git/evo_mwc_julia/tmp/bw.stan', line 83, column 2 to column 47)
If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or 

In [106]:
d = collect_params_from_chain(bw_names, bw_chains[:,:,1])

Dict{Any,Any} with 13 entries:
  "accept_stat__" => [0.857819; 1.0; … ; 0.951821; 0.880298]
  "sigma"         => [0.00313808; 0.00315847; … ; 0.00253087; 0.00308692]
  "divergent__"   => [0.0; 0.0; … ; 0.0; 0.0]
  "energy__"      => [-295.84; -290.797; … ; -294.589; -294.049]
  "alpha"         => [0.416855; 0.298272; … ; 0.485256; 0.668523]
  "g_predict"     => [1.80906e-5 -1.50629e-5 … 8.72128e-6 1.04867e-5; -8.38258e…
  "rho"           => [7595.96; 7272.46; … ; 6996.12; 7864.2]
  "treedepth__"   => [2.0; 3.0; … ; 3.0; 3.0]
  "n_leapfrog__"  => [7.0; 7.0; … ; 7.0; 7.0]
  "f_predict"     => [0.0370821 0.036377 … 0.824804 0.825506; 0.035682 0.036300…
  "lp__"          => [297.044; 294.283; … ; 295.286; 294.95]
  "stepsize__"    => [0.585573; 0.585573; … ; 0.585573; 0.585573]
  "y_predict"     => [0.0396156 0.0356201 … 0.826566 0.818884; 0.034153 0.03874…

In [107]:
sample_Stack = [d["y_predict"][i, :] for i in 1:100];
g_sample_Stack = [d["g_predict"][i, :] for i in 1:100];

In [108]:
predictive_regression(sample_Stack, range(minimum(t), stop=maximum(t), length=500), data=[t, N],kwargs=Dict{Any,Any}(:xlabel=>"time[s]", :ylabel=>"OD"))

<!DOCTYPE html>
 
 
 Plots.jl

In [109]:
predictive_regression(g_sample_Stack, range(minimum(t), stop=maximum(t), length=500),kwargs=Dict{Any,Any}(:xlabel=>"time[s]", :ylabel=>"Growth Rate[1/s]"))

<!DOCTYPE html>
 
 
 Plots.jl